[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/diffusers-image-outpaint-jupyter/blob/main/diffusers_image_outpaint_jupyter.ipynb)

In [ ]:
%cd /content
!git clone -b dev https://github.com/camenduru/diffusers-image-outpaint-hf
%cd /content/diffusers-image-outpaint-hf
!pip install transformers accelerate diffusers

import torch
from diffusers import AutoencoderKL
from diffusers.models.model_loading_utils import load_state_dict
from huggingface_hub import hf_hub_download

from controlnet_union import ControlNetModel_Union
from pipeline_fill_sd_xl import StableDiffusionXLFillPipeline

from PIL import Image, ImageDraw

config_file = hf_hub_download("xinsir/controlnet-union-sdxl-1.0", filename="config_promax.json")
config = ControlNetModel_Union.load_config(config_file)
controlnet_model = ControlNetModel_Union.from_config(config)
model_file = hf_hub_download("xinsir/controlnet-union-sdxl-1.0", filename="diffusion_pytorch_model_promax.safetensors")
state_dict = load_state_dict(model_file)
model, _, _, _, _ = ControlNetModel_Union._load_pretrained_model(controlnet_model, state_dict, model_file, "xinsir/controlnet-union-sdxl-1.0")
model.to(device="cuda", dtype=torch.float16)
vae = AutoencoderKL.from_pretrained("madebyollin/sdxl-vae-fp16-fix", torch_dtype=torch.float16).to("cuda")
pipe = StableDiffusionXLFillPipeline.from_pretrained("SG161222/RealVisXL_V5.0_Lightning", torch_dtype=torch.float16, vae=vae, controlnet=model, variant="fp16").to("cuda")

def infer(image, width, height, overlap_width, num_inference_steps, prompt_input=None):
    source = image
    target_size = (width, height)
    overlap = overlap_width

    if source.width < target_size[0] and source.height < target_size[1]:
        scale_factor = min(target_size[0] / source.width, target_size[1] / source.height)
        new_width = int(source.width * scale_factor)
        new_height = int(source.height * scale_factor)
        source = source.resize((new_width, new_height), Image.LANCZOS)

    if source.width > target_size[0] or source.height > target_size[1]:
        scale_factor = min(target_size[0] / source.width, target_size[1] / source.height)
        new_width = int(source.width * scale_factor)
        new_height = int(source.height * scale_factor)
        source = source.resize((new_width, new_height), Image.LANCZOS)

    margin_x = (target_size[0] - source.width) // 2
    margin_y = (target_size[1] - source.height) // 2

    background = Image.new('RGB', target_size, (255, 255, 255))
    background.paste(source, (margin_x, margin_y))

    mask = Image.new('L', target_size, 255)
    mask_draw = ImageDraw.Draw(mask)
    mask_draw.rectangle([
        (margin_x + overlap, margin_y + overlap),
        (margin_x + source.width - overlap, margin_y + source.height - overlap)
    ], fill=0)

    cnet_image = background.copy()
    cnet_image.paste(0, (0, 0), mask)

    final_prompt = "high quality"
    if prompt_input and prompt_input.strip():
        final_prompt += ", " + prompt_input

    (
        prompt_embeds,
        negative_prompt_embeds,
        pooled_prompt_embeds,
        negative_pooled_prompt_embeds,
    ) = pipe.encode_prompt(final_prompt, "cuda", True)

    results = []
    
    for image in pipe(
        prompt_embeds=prompt_embeds,
        negative_prompt_embeds=negative_prompt_embeds,
        pooled_prompt_embeds=pooled_prompt_embeds,
        negative_pooled_prompt_embeds=negative_pooled_prompt_embeds,
        image=cnet_image,
        num_inference_steps=num_inference_steps
    ):
        results.append((cnet_image, image))

    image = image.convert("RGBA")
    cnet_image.paste(image, (0, 0), mask)

    results.append((background, cnet_image))

    return results

In [ ]:
input_image = Image.open("/content/diffusers-image-outpaint-hf/examples/example_3.jpg")
width = 1280
height = 720
overlap_width = 42
num_inference_steps = 8
prompt_input = None
output_image = infer(input_image, height, width, overlap_width, num_inference_steps, None)
output_image[num_inference_steps+1][1]